In [0]:
# create autoloader_input folder in volume
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/01")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/02")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/03")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/04")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/05")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/06")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/07")

In [0]:
# create checkpoint location in Volume
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/checkpoint/autoloader")

In [0]:
# Copy Files to nested location
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv","/Volumes/dev/bronze/landing/autoloader_input/2010/12/01")
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-02.csv","/Volumes/dev/bronze/landing/autoloader_input/2010/12/02")
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-03.csv","/Volumes/dev/bronze/landing/autoloader_input/2010/12/03")

In [0]:
dbutils.fs.cp("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-05.csv","/Volumes/dev/bronze/landing/autoloader_input/2010/12/05")

In [0]:
# Read files using autoloader with checkpoint
# and schema location "/Volumes/dev/bronze/landing/checkpoint/autoloader"
# File Detection modes
# - Directory Listing (uses API calls to detect new files)
# - File notification ( uses notification and queue services - requires elevated cloud permission for setup)

df = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("pathGlobalFilter", "*.csv")
    .option("header", "true")
    .option("cloudFiles.schemaHints","Quantity int, UnitPrice double")
    .option("cloudFiles.schemaLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1/")
    .load("/Volumes/dev/bronze/landing/autoloader_input/*/")
)

In [0]:
#Write data to delta table - dev.bronze.invoice_al_1
from pyspark.sql.functions import col
(
    df.withColumn("_file",col("_metadata.file_name"))
    .writeStream
    .option("checkpointLocation", "/Volumes/dev/bronze/landing/checkpoint/autoloader/1/")
    .outputMode("append")
    .option("mergeSchema", "true")
    .trigger(availableNow=True)
    .toTable("dev.bronze.invoice_al_1")
)

In [0]:
%sql
select * from dev.bronze.invoice_al_1

In [0]:
%sql
select _file,count(1) 
from dev.bronze.invoice_al_1
group by _file